# <center> An introduction to distributed training<center>

Our computation resources have increased with multiple machines where each machine have multiple GPUs. Training a neural network on only 1 GPU does not make full use of our hardwares. In this note we will understand the mechanism of training on multiple gpus. The information across the internet is not complete, instead just copy paste the code to make things work, it is more important to understand what's happening behind the scene. In this way we can also get in touch with multiprocessing, which may also help to speedup scientific research.

We will begin with basic concepts of parallel programming, then talk about different set up to training neural networks on multiple gpus. The main topics are listed as follows:
- Threads and Processes.
- Problems might happened in parallel programming.
- Some example use cases of parallel programming for scientific computing.